In [4]:
import pickle
import os
import numpy as np
from random import randint
from sklearn.preprocessing import normalize
input_path1 ='/home/prithvi/Downloads/large_matrix_from_srinivas/main_x_list.p'
input_path2 ='/home/prithvi/Downloads/large_matrix_from_srinivas/main_lemma_list.p'
input_path3 ='/home/prithvi/Downloads/large_matrix_from_srinivas/main_cng_list.p'
input_path4 ='/home/prithvi/Downloads/large_matrix_from_srinivas/main_mod_list.p'
A=pickle.load(open("/home/prithvi/Downloads/large_matrix_from_srinivas/csr_sparse_matrix.p" , 'rb'))
A_normalized = normalize(A, norm='l1', axis=1)
main_x_list = pickle.load(open(input_path1, "rb"))
main_lemma_list = pickle.load(open(input_path2, "rb"))
main_cng_list= pickle.load(open(input_path3, "rb"))
main_mod_list = pickle.load(open(input_path4, "rb"))

In [29]:
import pickle
import os
input_path ='/home/prithvi/Downloads/lemmas_data'
files_1=set(sorted(os.listdir(input_path)))
files_1 =list(files_1)
d=[]
for i,each1 in enumerate(files_1):
    path = input_path+'/' +each1
    d.append((pickle.load(open(path, "rb"))))

In [3]:
input_path ='/home/prithvi/Downloads/lemmas_data_1'
files_1=set(sorted(os.listdir(input_path)))
files_1 =list(files_1)
d1=[]
for i,each1 in enumerate(files_1):
    path = input_path+'/' +each1
    d1.append((pickle.load(open(path, "rb"))))

In [7]:
input_path ='/home/prithvi/Downloads/intersect'
files_1=set(sorted(os.listdir(input_path)))
files_1 =list(files_1)
result=[]
for i,each1 in enumerate(files_1):
    path = input_path+'/' +each1
    result.append((pickle.load(open(path, "rb"))))

In [3]:
len(result)

4926

In [3]:
result[1]

[{u'lemmas': [[u'vac'], [u'na']], u'sentence': u'ucyate na   '},
 {u'keys': [[0, 4], [0]],
  u'lemmas': [[u'vac', u'uc'], [u'tad'], [u'na']],
  u'sentence': u'ucyate na   '}]

In [17]:
import pickle
import os
import numpy as np
from random import randint
import random
greedy_paths=[]
optimal_paths=[]
# input_path ='/home/prithvi/Downloads/extract/extras/all_dcs_lemmas_matrix.p'
# input_load = pickle.load(open(input_path, "rb"))
class beamsearch:
    def __init__(self,k,levels,di):
        self.k=k
        self.levels = levels  
        self.nodes_list=di
        self.k_paths_greedy=self.top_k_paths_greedy(self.nodes_list,k)
        self.k_paths_optimal,self.p_values=self.top_k_paths_optimal(self.nodes_list,k)
    def top_k_paths_greedy(self,l,k):
        s = [[] for _ in range(k)]
        for i in range(0,len(l)):
            if i==0:
                temp = []
                if(len(l[i])>k):
                    v = random.sample(range(0,len(l[i])),k)
                    for j in v:
                        temp.append([l[i][j]])
                else:
                    for j in range(0,len(l[i])):
                        temp.append([l[i][j]])
                s=temp
            else:
                count = 0
                for m in s:
                    if m:
                        count+=1
                t = np.zeros(shape=(count,len(l[i]))) 
                for p in range(0,t.shape[0]):
                    for q in range(0,t.shape[1]):
                        t[p,q]=A_normalized[s[p][-1],l[i][q]]
                t_1d = t.flatten()
                idx_1d = t_1d.argsort()[-k:][::-1]
                x_idx, y_idx = np.unravel_index(idx_1d, t.shape)
                temp = []
                for x, y, in zip(x_idx, y_idx):
                    temp.append(s[x]+[l[i][y]])
                s = temp
        return s
    def top_k_paths_optimal(self,l,k):
        s = [[] for _ in range(k)]
        p_values=[]
        for i in range(0,len(l)):
            if i==0:
                temp = []
                if len(l[i])>k:
                    v = random.sample(range(0,len(l[i])),k)
                    for j in v:
                        temp.append([l[i][j]])
                else:
                    for j in range(0,len(l[i])):
                        temp.append([l[i][j]])
                p_values = [0.5]*len(temp)
                s=temp
            else:
                count = 0
                for m in s:
                    if m:
                        count+=1
                t = np.zeros(shape=(count,len(l[i]))) 
                for p in range(0,t.shape[0]):
                    for q in range(0,t.shape[1]):
                        t[p,q]=A_normalized[s[p][-1],l[i][q]]
                t = (t.T * p_values).T
                t_1d = t.flatten()
                idx_1d = t_1d.argsort()[-k:][::-1]
                x_idx, y_idx = np.unravel_index(idx_1d, t.shape)
                temp = []
                for x, y, in zip(x_idx, y_idx):
                    temp.append(s[x]+[l[i][y]])
                p_values = [t[x][y] for x, y, in zip(x_idx, y_idx)]
                s = temp
        return s,p_values        
def main():
    for i in result:
        di=[]
        #print i['sentence']
        #print i['keys']
        #print ("Lemmas list")
        #print i['lemmas']
        #print i[1]['lemmas']
        for j in i[1]['lemmas']:
            l=[]
            for k in j:
                if(k.encode("ascii","ignore") in main_x_list):
                    l.append(main_x_list.index(k.encode("ascii","ignore")))
                else:
                    pass
            di.append(l)
        di = [x for x in di if x != []]
        k=2
        levels=len(di)
        bs = beamsearch(k,levels,di)
        g=[]
        for i in bs.k_paths_greedy:
            greedy_path=[main_x_list[x] for x in i]
            g.append(greedy_path)
        greedy_paths.append(g)
        o=[]
        for i in bs.k_paths_optimal:
            optimal_path=[main_x_list[x] for x in i]
            o.append(optimal_path)
        optimal_paths.append(o)
if __name__ == "__main__": main()   

In [18]:
actual_paths=[]
for i in result:
    p=[]
    for j in i[0]['lemmas']:
        for k in j:
            p.append(k)
    actual_paths.append(p)
        

In [30]:
from __future__ import division
count=0
count1=0
count2=0
total=0
a=[]
for i in range(0,len(optimal_paths)):
    if len(actual_paths[i]) == len(optimal_paths[i][0]) or len(actual_paths[i]) == len(optimal_paths[i][0]):
        total+=1
        if optimal_paths[i][0]==actual_paths[i]:
            count+=1
            count1+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(optimal_paths[i][0])
            count2+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(actual_paths[i])
        else:
            try:
                if optimal_paths[i][1] == actual_paths[i]:
                    count+=1
                    count1+=len(list(set(actual_paths[i]) & set(optimal_paths[i][1])))/len(optimal_paths[i][1])
                    count2+=len(list(set(actual_paths[i]) & set(optimal_paths[i][1])))/len(actual_paths[i])
                else:
                    count1+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(optimal_paths[i][0])
                    count2+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(actual_paths[i])
            except:
                count1+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(optimal_paths[i][0])
                count2+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(actual_paths[i])
    else:
        count1+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(optimal_paths[i][0])
        count2+=len(list(set(actual_paths[i]) & set(optimal_paths[i][0])))/len(actual_paths[i])
        a.append([actual_paths[i],optimal_paths[i][0]])
        
        


In [31]:
count/total

0.7350427350427351

In [35]:
count1/len(actual_paths)

0.6198249855355993

In [18]:
input_path ='/home/prithvi/Downloads/NLP Project/SKT_File/extras/sandhiRules.p'
pickle.load(open(input_path, "rb"))

ValueError: unsupported pickle protocol: 3

In [2]:
import csv
for line in open('/home/prithvi/Downloads/movies.txt'):
    rp=[line.split(".json")[0]]
    with open('imdb_movies.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(rp)
    